In [1]:
!pip install pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum as sum_, max as max_, min as min_


In [3]:
spark = SparkSession.builder.appName("Online Banking Analysis").getOrCreate()

In [4]:
from google.colab import files
uploaded = files.upload()


Saving credit card.csv to credit card.csv
Saving loan.csv to loan.csv
Saving txn.csv to txn.csv


In [5]:
loan_df = spark.read.csv("/content/loan.csv", header=True, inferSchema=True)
credit_df = spark.read.csv("/content/credit card.csv", header=True, inferSchema=True)
txn_df = spark.read.csv("/content/txn.csv", header=True, inferSchema=True)

In [6]:
loan_df.printSchema()
credit_df.printSchema()
txn_df.printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Family Size: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Expenditure: integer (nullable = true)
 |-- Use Frequency: integer (nullable = true)
 |-- Loan Category: string (nullable = true)
 |-- Loan Amount: string (nullable = true)
 |-- Overdue: integer (nullable = true)
 |--  Debt Record: string (nullable = true)
 |--  Returned Cheque: integer (nullable = true)
 |--  Dishonour of Bill: integer (nullable = true)

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: dou

In [7]:
loan_df.show(5)
credit_df.show(5)
txn_df.show(5)

+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|  Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+------------+--------------+-----------+------+-----------+-------------+-------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|      HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|   PROFESSOR|       MARRIED|          6| 51000|      19999|            4|     SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|     DENTIST|        SINGLE|          3| 58450|      27675|            

In [9]:
#Loan data use cases

print("\n# Number of loans in each category:")
loan_df.groupBy("Loan Category").count().show()



# Number of loans in each category:
+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



In [10]:
print("\n# Number of people who have taken more than 1 lakh loan:")
loan_df.filter(col("Loan Amount") > 100000).select("Customer_ID").distinct().count()


# Number of people who have taken more than 1 lakh loan:


0

In [11]:
print("\n# Number of people with income greater than 60000 rupees:")
loan_df.filter(col("Income") > 60000).select("Customer_ID").distinct().count()


# Number of people with income greater than 60000 rupees:


198

In [13]:
print("\n# Number of people with 2 or more returned cheques and income less than 50000:")
loan_df.filter((col(" Returned Cheque") >= 2) & (col("Income") < 50000)).select("Customer_ID").distinct().count()



# Number of people with 2 or more returned cheques and income less than 50000:


136

In [14]:
print("\n# Number of people with 2 or more returned cheques and are single:")
loan_df.filter((col(" Returned Cheque") >= 2) & (col("Marital Status") == "Single")).select("Customer_ID").distinct().count()


# Number of people with 2 or more returned cheques and are single:


0

In [15]:
print("\n# Number of people with expenditure over 50000 a month:")
loan_df.filter(col("Expenditure") > 50000).select("Customer_ID").distinct().count()


# Number of people with expenditure over 50000 a month:


6

In [16]:
print("\n# Number of members likely eligible for credit card (Income > 50k, Returned Cheque = 0):")
loan_df.filter((col("Income") > 50000) & (col(" Returned Cheque") == 0)) \
       .select("Customer_ID").distinct().count()



# Number of members likely eligible for credit card (Income > 50k, Returned Cheque = 0):


22

In [17]:
# Credit card use cases
print("\n# Credit card users in Spain:")
credit_df.filter(col("Geography") == "Spain").select("CustomerId").distinct().count()


# Credit card users in Spain:


2477

In [18]:
print("\n# Number of members who are likely eligible (CreditScore > 650) and active in the bank:")
credit_df.filter((col("CreditScore") > 650) & (col("IsActiveMember") == 1)) \
         .select("CustomerId").distinct().count()



# Number of members who are likely eligible (CreditScore > 650) and active in the bank:


2655

In [20]:
from pyspark.sql.functions import col, max as max_, min as min_, sum as sum_, to_date

In [21]:
txn_df = txn_df.toDF(*[c.strip().replace(" ", "_") for c in txn_df.columns])

In [22]:
txn_df = txn_df.withColumn("WITHDRAWAL_AMT", col("WITHDRAWAL_AMT").cast("double")) \
               .withColumn("DEPOSIT_AMT", col("DEPOSIT_AMT").cast("double")) \
               .withColumn("BALANCE_AMT", col("BALANCE_AMT").cast("double")) \
               .withColumn("VALUE_DATE", to_date("VALUE_DATE", "dd-MM-yyyy"))

In [23]:
print("\n# Maximum withdrawal amount in transactions:")
txn_df.select(max_("WITHDRAWAL_AMT").alias("Max_Withdrawal")).show()


# Maximum withdrawal amount in transactions:
+--------------+
|Max_Withdrawal|
+--------------+
| 4.594475464E8|
+--------------+



In [24]:
print("\n# Minimum withdrawal amount in transactions:")
txn_df.select(min_("WITHDRAWAL_AMT").alias("Min_Withdrawal")).show()


# Minimum withdrawal amount in transactions:
+--------------+
|Min_Withdrawal|
+--------------+
|          0.01|
+--------------+



In [25]:

print("\n# Maximum deposit amount in transactions:")
txn_df.select(max_("DEPOSIT_AMT").alias("Max_Deposit")).show()


# Maximum deposit amount in transactions:
+-----------+
|Max_Deposit|
+-----------+
|    5.448E8|
+-----------+



In [26]:
print("\n# Minimum deposit amount in transactions:")
txn_df.select(min_("DEPOSIT_AMT").alias("Min_Deposit")).show()


# Minimum deposit amount in transactions:
+-----------+
|Min_Deposit|
+-----------+
|       0.01|
+-----------+



In [27]:
print("\n# Sum of balance in every bank account:")
txn_df.groupBy("Account_No").agg(sum_("BALANCE_AMT").alias("Total_Balance")).show()


# Sum of balance in every bank account:
+-------------+--------------------+
|   Account_No|       Total_Balance|
+-------------+--------------------+
|409000438611'|-2.49486577068339...|
|     1196711'|-1.60476498101275E13|
|     1196428'| -8.1418498130721E13|
|409000493210'|-3.27584952132095...|
|409000611074'|       1.615533622E9|
|409000425051'|-3.77211841164998...|
|409000405747'|-2.43108047067000...|
|409000362497'| -5.2860004792808E13|
|409000493201'|1.0420831829499985E9|
|409000438620'|-7.12291867951358...|
+-------------+--------------------+



In [28]:
print("\n# Number of transactions on each date:")
txn_df.groupBy("VALUE_DATE").count().orderBy("VALUE_DATE").show()


# Number of transactions on each date:
+----------+------+
|VALUE_DATE| count|
+----------+------+
|      NULL|116201|
+----------+------+



In [29]:
print("\n# List of accounts with withdrawal amount more than 1 lakh:")
txn_df.filter(col("WITHDRAWAL_AMT") > 100000) \
      .select("Account_No", "WITHDRAWAL_AMT").distinct().show()


# List of accounts with withdrawal amount more than 1 lakh:
+-------------+--------------+
|   Account_No|WITHDRAWAL_AMT|
+-------------+--------------+
|409000611074'|      274600.0|
|409000493201'|     1500000.0|
|409000493201'|     199604.27|
|409000438620'|      186604.0|
|409000438620'|   3.6675558E7|
|     1196711'|     7530283.0|
|     1196428'|      812361.0|
|     1196428'|     6348768.0|
|     1196428'|    3043151.63|
|409000362497'|      576954.0|
|409000362497'|     3423962.0|
|409000362497'| 3.144482503E7|
|     1196428'|    4441827.47|
|409000611074'|      145450.0|
|409000493201'|     119401.28|
|     1196711'|      628945.0|
|     1196428'|     289670.04|
|409000362497'| 3.483281361E7|
|409000362497'| 4.289763641E7|
|409000362497'| 2.678162613E7|
+-------------+--------------+
only showing top 20 rows

